<h1><Center>Fetch Tract Numbers for the Crime Dataset</Center></h1>

This Notebook is to find tract numbers for the Crime Dataset via api.census.gov/data by creating an
account, getting an API key and calling the API to get the Tract data.

Data source: Crime Data: https://data.lacity.org/A-Safe-City/Crime-Data-from-2010-to-Present/63jg-8b9z

The Crime Dataset is huge and fetching tract numbers for the complete file at once took a lot of time. So the crime dataset is divided by years forming 7 files for the year 2010-2016. 

This part of the code is for the Crime Dataset for the year 2010. The same codecan be applied to the datasets from 2011-2016 as well by simply changing the read file.

Once the tracks were fetched, all the separate year files were combined to form a complete Crime Dataset, with Tract numbers.



In [33]:
#Import all the necessary packages
import subprocess
import os
import math
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import requests
import glob, os
from statsmodels.distributions.empirical_distribution import ECDF
from scipy.stats import mannwhitneyu

In [42]:
#Specify the Chunk Size
process_chunksize = 1000

#Read the File for the Year 2010
mainDF = pd.read_csv('/Users/aagnakatrodia/Desktop/GitAdd/Data_Mining/Files/InputYearCrime/2010.csv', chunksize=process_chunksize)
print(mainDF)


In [43]:
#Specify the api URL and Key and send a request command
census_api_url='https://api.census.gov/data/2017/acs/acs5?key=e335bd7f54d58725613d2bc9b7992f7d66313f86&get=NAME,B00001_001E,B19019_001E,B19083_001E&for=TRACT:*&in=state:06+COUNTY:037'
responseCensus = requests.get(census_api_url, headers={'Content-Type': 'application/json'})

In [44]:
#Load response content in textCensus
textCensus=json.loads(responseCensus.content)
tractDF = pd.DataFrame(textCensus)
print('Shape of census tract file: ' + str(tractDF.shape))
tractDF.head()

Shape of census tract file: (2347, 7)


,0,1,2,3,4,5,6
0,NAME,B00001_001E,B19019_001E,B19083_001E,state,county,tract
1,"Census Tract 5315.04, Los Angeles County, Cali...",226,41476,0.3636,06,037,531504
2,"Census Tract 5315.03, Los Angeles County, Cali...",266,37022,0.3241,06,037,531503
3,"Census Tract 2060.31, Los Angeles County, Cali...",272,87500,0.5100,06,037,206031
4,"Census Tract 1863.01, Los Angeles County, Cali...",347,38672,0.4834,06,037,186301


In [45]:
#Set the first row as column names
tractDF.rename(columns = tractDF.iloc[0], inplace=True)
tractDF.head()

#Remove the first row
tractDF = tractDF.iloc[1:]
tractDF.head()

,NAME,B00001_001E,B19019_001E,B19083_001E,state,county,tract
1,"Census Tract 5315.04, Los Angeles County, Cali...",226,41476,0.3636,06,037,531504
2,"Census Tract 5315.03, Los Angeles County, Cali...",266,37022,0.3241,06,037,531503
3,"Census Tract 2060.31, Los Angeles County, Cali...",272,87500,0.5100,06,037,206031
4,"Census Tract 1863.01, Los Angeles County, Cali...",347,38672,0.4834,06,037,186301
5,"Census Tract 1831.04, Los Angeles County, Cali...",290,69250,0.4169,06,037,183104


In [46]:
#Rename Columns, Remove unwanted columns, Change Column Type
tractDF.rename(index=str,columns={'B00001_001E':'Pop','B19019_001E':'Income','B19083_001E':'Gini','tract':'Tract'}, inplace=True)
tractDF.set_index('Tract', inplace=True)
tractDF.drop(['NAME', 'state', 'county'],inplace=True, axis=1)
tractDF=tractDF[['Pop','Income','Gini']].apply(pd.to_numeric, errors='coerce')
print(tractDF.info())
print(tractDF.head(8))

<class 'pandas.core.frame.DataFrame'>
Index: 2346 entries, 531504 to 602002
Data columns (total 3 columns):
Pop       2336 non-null float64
Income    2346 non-null int64
Gini      2346 non-null float64
dtypes: float64(2), int64(1)
memory usage: 73.3+ KB
None
          Pop     Income          Gini
Tract                                 
531504  226.0      41476  3.636000e-01
531503  266.0      37022  3.241000e-01
206031  272.0      87500  5.100000e-01
186301  347.0      38672  4.834000e-01
183104  290.0      69250  4.169000e-01
311801  230.0      58199  3.949000e-01
980028    0.0 -666666666 -6.666667e+08
980026    NaN -666666666  1.688000e-01


In [47]:
#GetTract Function
def getTract(lonlat):
    tract = -999
    api_url='https://geo.fcc.gov/api/census/area?lat='+str(lonlat[0])+'&lon='+str(lonlat[1])+'&format=json'
    headers = {'Content-Type': 'application/json'}
    response = requests.get(api_url, headers=headers)
    try:
        tract=json.loads(response.content.decode('utf-8'))['results'][0]['block_fips'][5:11]
    except:
        print('No Tract found from FCC geo API')
    return tract

In [48]:
#Initialize Variables
df = pd.DataFrame()
start = 0
end = 1

In [50]:
for mainDF_chunk in mainDF:
    First = start * process_chunksize
    Last = end * process_chunksize
    print('New group from main DF, lines ' + str(First) + ' to ' + str(Last))

    ## Cleaning main-------------------------------------------------------------
    #Remove Trailing spaces
    mainDF_chunk.rename(columns=lambda x: x.replace(' ', ''), inplace=True)
    
    #Edit Location
    mainDF_chunk.dropna(subset=['Location'], inplace=True)
    mainDF_chunk['Location'] = mainDF_chunk['Location'].map(lambda x: eval(str(x)))
    
    #Change Type to DateTime
    mainDF_chunk[['DateReported', 'DateOccurred', 'TimeOccurred']].apply(pd.to_datetime, errors='coerce')
    
    #Change Type to Numeric
    mainDF_chunk[['DRNumber', 'AreaID', 'ReportingDistrict', 'CrimeCode', 'VictimAge', 'PremiseCode', 'WeaponUsedCode',
                  'CrimeCode1']].apply(pd.to_numeric)
    
    #Print Dimension
    print('Dimension of main DF group:' + str(mainDF_chunk.shape))

    # Add Tract from FCC geo AND violent bool
    mainDF_chunk['Tract'] = mainDF_chunk.Location.map(lambda x: getTract(x))

    #Merge MainDF and TractDF to get the Tracts
    df_group = pd.merge(mainDF_chunk, tractDF, how='left', on='Tract', sort=True)
    
    #Keep appending it to DF
    df = df.append(df_group, sort=True)

    start += 1
    end += 1

New group from main DF, lines 0 to 1000
Dimension of main DF group:(1000, 29)
New group from main DF, lines 1000 to 2000
Dimension of main DF group:(1000, 29)
New group from main DF, lines 2000 to 3000
Dimension of main DF group:(1000, 29)
New group from main DF, lines 3000 to 4000
Dimension of main DF group:(1000, 29)
No Tract found from FCC geo API
No Tract found from FCC geo API
New group from main DF, lines 4000 to 5000
Dimension of main DF group:(1000, 29)
New group from main DF, lines 5000 to 6000
Dimension of main DF group:(1000, 29)
New group from main DF, lines 6000 to 7000
Dimension of main DF group:(1000, 29)
New group from main DF, lines 7000 to 8000
Dimension of main DF group:(1000, 29)
New group from main DF, lines 8000 to 9000
Dimension of main DF group:(1000, 29)
New group from main DF, lines 9000 to 10000
Dimension of main DF group:(1000, 29)
New group from main DF, lines 10000 to 11000
Dimension of main DF group:(1000, 29)
New group from main DF, lines 11000 to 12000


Dimension of main DF group:(1000, 29)
New group from main DF, lines 98000 to 99000
Dimension of main DF group:(1000, 29)
New group from main DF, lines 99000 to 100000
Dimension of main DF group:(1000, 29)
New group from main DF, lines 100000 to 101000
Dimension of main DF group:(1000, 29)
New group from main DF, lines 101000 to 102000
Dimension of main DF group:(1000, 29)
New group from main DF, lines 102000 to 103000
Dimension of main DF group:(1000, 29)
New group from main DF, lines 103000 to 104000
Dimension of main DF group:(1000, 29)
New group from main DF, lines 104000 to 105000
Dimension of main DF group:(1000, 29)
New group from main DF, lines 105000 to 106000
Dimension of main DF group:(1000, 29)
New group from main DF, lines 106000 to 107000
Dimension of main DF group:(1000, 29)
New group from main DF, lines 107000 to 108000
Dimension of main DF group:(1000, 29)
New group from main DF, lines 108000 to 109000
Dimension of main DF group:(1000, 29)
New group from main DF, lines 

New group from main DF, lines 192000 to 193000
Dimension of main DF group:(1000, 29)
New group from main DF, lines 193000 to 194000
Dimension of main DF group:(1000, 29)
No Tract found from FCC geo API
New group from main DF, lines 194000 to 195000
Dimension of main DF group:(1000, 29)
New group from main DF, lines 195000 to 196000
Dimension of main DF group:(1000, 29)
New group from main DF, lines 196000 to 197000
Dimension of main DF group:(1000, 29)
New group from main DF, lines 197000 to 198000
Dimension of main DF group:(1000, 29)
New group from main DF, lines 198000 to 199000
Dimension of main DF group:(1000, 29)
New group from main DF, lines 199000 to 200000
Dimension of main DF group:(1000, 29)
No Tract found from FCC geo API
New group from main DF, lines 200000 to 201000
Dimension of main DF group:(1000, 29)
New group from main DF, lines 201000 to 202000
Dimension of main DF group:(1000, 29)


In [51]:
df.head()

,Address,AreaID,AreaName,CrimeCode,CrimeCode1,CrimeCodeDescription,CrossStreet,DRNumber,DateOccurred,DateReported,...,StatusDescription,TimeOccurred,Tract,Unnamed:0,VictimAge,VictimDescent,VictimSex,WeaponDescription,WeaponUsedCode,Year_Occur
0,7400 SUNNYBRAE AV,21.0,Topanga,210,210.0,ROBBERY,NaN,102113132,06/21/2010,06/21/2010,...,Invest Cont,2240,113101,0,32.0,H,M,SEMI-AUTOMATIC PISTOL,109.0,2010
1,8300 NATALIE LN,21.0,Topanga,624,624.0,BATTERY - SIMPLE ASSAULT,NaN,102114593,07/20/2010,07/20/2010,...,Invest Cont,2350,113231,1,54.0,H,F,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",400.0,2010
2,8300 LESTER LN,21.0,Topanga,420,420.0,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),NaN,102112077,06/01/2010,06/01/2010,...,Invest Cont,1145,113231,2,28.0,W,M,NaN,NaN,2010
3,8800 HILLCROFT TL,21.0,Topanga,649,649.0,DOCUMENT FORGERY / STOLEN FELONY,NaN,102113030,06/16/2010,06/19/2010,...,Invest Cont,1410,113231,3,46.0,W,F,NaN,NaN,2010
4,8500 FAUST AV,21.0,Topanga,624,624.0,BATTERY - SIMPLE ASSAULT,NaN,102113474,06/26/2010,06/28/2010,...,Invest Cont,1000,113232,4,15.0,W,F,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",400.0,2010


In [53]:
#Save the data frame to a CSV file
df.to_csv("/Users/aagnakatrodia/Desktop/GitAdd/Data_Mining/Files/Outputs/Tract2010.csv", sep=',', encoding='utf-8', header = True)